## Imports and Setup

In [1]:
import os
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import sys
sys.path.insert(0, "../src")
import sat_download

sys.path.insert(0, "../utils/")
import data_utils
import config_utils
import pred_utils
import embed_utils

%load_ext autoreload
%autoreload 2

## Load Config Files

In [2]:
iso_code = "ZWE"
adm_level = "ADM2"
sum_threshold = 5
spacing = 150
buffer_size = 150

cwd = os.path.dirname(os.getcwd())
data_config_file = os.path.join(cwd, "configs/data_configs/data_config_ISO_AF.yaml")
data_config = config_utils.load_config(data_config_file)

sat_config_file = os.path.join(cwd, "configs/sat_configs/sat_config_500x500_60cm.yaml")
sat_creds_file = os.path.join(cwd, "configs/sat_configs/issa_sat_creds.yaml")

sat_config = config_utils.load_config(sat_config_file)
sat_creds = config_utils.create_config(sat_creds_file)

## Generate Prediction Tiles

In [3]:
geoboundary = data_utils._get_geoboundaries(data_config, iso_code, adm_level=adm_level)
for i, shapename in enumerate(geoboundary.shapeName.unique()):
    tiles = pred_utils.generate_pred_tiles(
        data_config, iso_code, spacing, buffer_size, adm_level, shapename
    ).reset_index(drop=True)
print(f"Total tiles: {tiles.shape}")
tiles.head(3)

Total tiles: (7282, 4)


,UID,shapeName,sum,geometry
0,582,Harare Rural,2.0,"POLYGON ((3439360.595 -2031495.512, 3439360.59..."
1,583,Harare Rural,2.0,"POLYGON ((3439360.595 -2031345.512, 3439360.59..."
2,768,Harare Rural,9.0,"POLYGON ((3439660.595 -2030595.512, 3439660.59..."


## Download Satellite Images

In [ ]:
geoboundary = data_utils._get_geoboundaries(data_config, iso_code, adm_level="ADM2")
for i, shapename in enumerate(geoboundary.shapeName.unique()):
    print(f"{shapename} {i}/{len(geoboundary.shapeName.unique())}")
    tiles = pred_utils.generate_pred_tiles(
        data_config, iso_code, spacing, buffer_size, adm_level, shapename
    ).reset_index(drop=True)
    tiles["points"] = tiles["geometry"].centroid
    tiles = tiles[tiles["sum"] > sum_threshold].reset_index(drop=True)
    print(f"Total tiles: {tiles.shape}")
    
    data = tiles.copy()
    data["geometry"] = data["points"]
    sat_dir = os.path.join(cwd, "output", iso_code, "images", shapename)
    sat_download.download_sat_images(sat_creds, sat_config, data=data, out_dir=sat_dir)

## CNN Model

In [ ]:
model_config_file = os.path.join(cwd, f"configs/cnn_configs/vgg16.yaml")
model_config = config_utils.load_config(model_config_file)
result = pred_utils.cnn_predict(tiles, iso_code, shapename, model_config, sat_dir, n_classes=2)

## ViT Model

In [13]:
model_config_file = os.path.join(cwd, f"configs/model_configs/dinov2_vitl14-LR.yaml")
model_config = config_utils.load_config(model_config_file)
result = pred_utils.vit_pred(tiles, model_config, iso_code, shapename, sat_dir)

Using cache found in /home/itingzon.unicef/.cache/torch/hub/facebookresearch_dinov2_main
INFO:dinov2:using MLP layer as FFN
INFO:root:Device: cpu
INFO:root:Generating embeddings...
INFO:root:Reading file /home/itingzon.unicef/giga/output/SEN/embeddings/SEN_Foundiougne_dinov2_vitl14_embeds.csv
INFO:root:Loaded /home/itingzon.unicef/giga/exp/SEN-dinov2_vitl14-LR/SEN-dinov2_vitl14-LR.pkl
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex:sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:root:Generated /home/itingzon.unicef/giga/output/SEN/results/SEN_Foundiougne_dinov2_vitl14-LR_results.gpkg
